In [2]:
setwd(paste0(Sys.getenv('R_SOURCES'), '/house_prices'))

house_prices <- source('main.R', local = TRUE)$value

################################
# load data, missing, outliers #
################################
combined_dataset_step1 <- 
    house_prices$helpers$load_data() %>%
    (house_prices$stage1_transformation)

################################
# transform numeric predictors #
################################
step2 <- house_prices$stage2_transformation(
    combined_dataset_step1,
    threshold = 20
)
trans_config <- step2$trans_config
combined_dataset_step2 <- step2$dataset
                
trans_config

house_prices$helpers$utils$frames_diff(
    combined_dataset_step1, 
    combined_dataset_step2
) %>% head

####################################                
# transform categorical predictors #
####################################
combined_dataset_step3 <- house_prices$stage3_transformation(combined_dataset_step2)

house_prices$helpers$utils$frames_diff(
    combined_dataset_step2, 
    combined_dataset_step3
) %>% head

var,tran_name,progress_score,tran_fn,r2_x,r2_tran
GrLivArea,log,91.15794,"function (x) , log(x + 1)",0.52593095,0.5438044
LotArea,inv4,78.86403,"function (x) , x^(1/4)",0.06788313,0.1591873
TotRmsAbvGrd,log,30.68502,"function (x) , log(x + 1)",0.28912299,0.2938676


GrLivArea.1,GrLivArea.2,LotArea.1,LotArea.2,TotRmsAbvGrd.1,TotRmsAbvGrd.2
1710,7.444833,8450,9.587694,8,2.197225
1262,7.141245,9600,9.898464,6,1.945910
1786,7.488294,11250,10.298836,6,1.945910
1717,7.448916,9550,9.885550,7,2.079442
2198,7.695758,14260,10.927728,9,2.302585
1362,7.217443,14115,10.899842,5,1.791759


Alley.1,Alley.2,BldgType.1,BldgType.2,BsmtCond.1,BsmtCond.2,BsmtExposure.1,BsmtExposure.2,BsmtFinType1.1,BsmtFinType1.2,⋯,RoofStyle.1,RoofStyle.2,SaleCondition.1,SaleCondition.2,SaleType.1,SaleType.2,Street.1,Street.2,Utilities.1,Utilities.2
_none_,2.531822,1Fam,2.564039,TA,2.540871,No,2.320042,GLQ,3.262019,⋯,Gable,2.418054,Normal,2.451586,WD,2.421468,Pave,2.499311,AllPub,2.496911
_none_,2.531822,1Fam,2.564039,TA,2.540871,Gd,3.295455,ALQ,2.231818,⋯,Gable,2.418054,Normal,2.451586,WD,2.421468,Pave,2.499311,AllPub,2.496911
_none_,2.531822,1Fam,2.564039,TA,2.540871,Mn,2.675439,GLQ,3.262019,⋯,Gable,2.418054,Normal,2.451586,WD,2.421468,Pave,2.499311,AllPub,2.496911
_none_,2.531822,1Fam,2.564039,Gd,3.046154,No,2.320042,ALQ,2.231818,⋯,Gable,2.418054,Abnorml,1.920792,WD,2.421468,Pave,2.499311,AllPub,2.496911
_none_,2.531822,1Fam,2.564039,TA,2.540871,Av,2.900452,GLQ,3.262019,⋯,Gable,2.418054,Normal,2.451586,WD,2.421468,Pave,2.499311,AllPub,2.496911
_none_,2.531822,1Fam,2.564039,TA,2.540871,No,2.320042,GLQ,3.262019,⋯,Gable,2.418054,Normal,2.451586,WD,2.421468,Pave,2.499311,AllPub,2.496911


In [19]:
df <- combined_dataset_step3 %>% select(-Id, -price_log, -dataSource)

M <- abs(cor(df))
diag(M) <- 0
M <- as_data_frame(M)

cols <- purrr::map_lgl(M, function(col) { any(col > 0.8) })

M2 <- abs(cor(df[names(which(cols))]))
diag(M2) <- 0

M2

,Exterior1st,Exterior2nd,FireplaceQu,Fireplaces,GarageArea,GarageCars,GarageCond,GarageQual,GarageYrBlt,GrLivArea,SaleCondition,SaleType,TotRmsAbvGrd
Exterior1st,0.0000000,0.91511234,0.1716986,0.11017091,0.3281537,0.3693900,0.15350436,0.19229995,0.12551996,0.2365318,0.28258081,0.27561928,0.16269785
Exterior2nd,0.9151123,0.00000000,0.1620534,0.09545221,0.3271569,0.3638656,0.13089176,0.16183800,0.10326690,0.2275226,0.28821724,0.29169891,0.16329625
FireplaceQu,0.1716986,0.16205344,0.0000000,0.84656470,0.3416352,0.3840579,0.21675681,0.21480948,0.20190266,0.5102708,0.15436100,0.14769380,0.36065534
Fireplaces,0.1101709,0.09545221,0.8465647,0.00000000,0.2910259,0.3222712,0.21370826,0.21180922,0.18669012,0.4623676,0.08760824,0.08825245,0.31326530
GarageArea,0.3281537,0.32715690,0.3416352,0.29102591,0.0000000,0.8922891,0.50500904,0.51988049,0.55086048,0.4827861,0.28514096,0.26374334,0.32875182
GarageCars,0.3693900,0.36386563,0.3840579,0.32227122,0.8922891,0.0000000,0.51965527,0.52760336,0.58051960,0.5097740,0.29188057,0.26941526,0.36329117
GarageCond,0.1535044,0.13089176,0.2167568,0.21370826,0.5050090,0.5196553,0.00000000,0.80720089,0.82425042,0.1889460,0.13373071,0.09395077,0.08380553
GarageQual,0.1923000,0.16183800,0.2148095,0.21180922,0.5198805,0.5276034,0.80720089,0.00000000,0.79802674,0.1966582,0.13942478,0.09045071,0.08876719
GarageYrBlt,0.1255200,0.10326690,0.2019027,0.18669012,0.5508605,0.5805196,0.82425042,0.79802674,0.00000000,0.1746723,0.13382785,0.08048634,0.07934761
GrLivArea,0.2365318,0.22752262,0.5102708,0.46236761,0.4827861,0.5097740,0.18894597,0.19665819,0.17467233,0.0000000,0.17011728,0.18086532,0.80924237


In [20]:
apply(M2, 2, function(c) { c > 0.8  })

,Exterior1st,Exterior2nd,FireplaceQu,Fireplaces,GarageArea,GarageCars,GarageCond,GarageQual,GarageYrBlt,GrLivArea,SaleCondition,SaleType,TotRmsAbvGrd
Exterior1st,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
Exterior2nd,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
FireplaceQu,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
Fireplaces,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
GarageArea,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
GarageCars,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
GarageCond,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,TRUE,FALSE,FALSE,FALSE,FALSE
GarageQual,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
GarageYrBlt,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE
GrLivArea,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE


In [37]:
house_prices <- source('main.R', local = TRUE)$value

################################
# load data, missing, outliers #
################################
combined_dataset_step1A <- 
    house_prices$helpers$load_data() %>%
    (house_prices$stage1_transformation) %>%
    select(-Exterior2nd, -FireplaceQu, -GarageCars, -GarageQual, -GarageYrBlt, -SaleType, -TotRmsAbvGrd)

################################
# transform numeric predictors #
################################
step2 <- house_prices$stage2_transformation(
    combined_dataset_step1A,
    threshold = 20
)
trans_configA <- step2$trans_config
combined_dataset_step2A <- step2$dataset
              
trans_configA

house_prices$helpers$utils$frames_diff(
    combined_dataset_step1A, 
    combined_dataset_step2A
) %>% head

####################################                
# transform categorical predictors #
####################################
combined_dataset_step3A <- house_prices$stage3_transformation(combined_dataset_step2A)
house_prices$helpers$utils$frames_diff(
    combined_dataset_step2A, 
    combined_dataset_step3A
) %>% head

var,tran_name,progress_score,tran_fn,r2_x,r2_tran
GrLivArea,log,91.15794,"function (x) , log(x + 1)",0.52593095,0.5438044
LotArea,inv4,78.86403,"function (x) , x^(1/4)",0.06788313,0.1591873


GrLivArea.1,GrLivArea.2,LotArea.1,LotArea.2
1710,7.444833,8450,9.587694
1262,7.141245,9600,9.898464
1786,7.488294,11250,10.298836
1717,7.448916,9550,9.885550
2198,7.695758,14260,10.927728
1362,7.217443,14115,10.899842


Alley.1,Alley.2,BldgType.1,BldgType.2,BsmtCond.1,BsmtCond.2,BsmtExposure.1,BsmtExposure.2,BsmtFinType1.1,BsmtFinType1.2,⋯,RoofMatl.1,RoofMatl.2,RoofStyle.1,RoofStyle.2,SaleCondition.1,SaleCondition.2,Street.1,Street.2,Utilities.1,Utilities.2
_none_,2.531822,1Fam,2.564039,TA,2.540871,No,2.320042,GLQ,3.262019,⋯,CompShg,2.483601,Gable,2.418054,Normal,2.451586,Pave,2.499311,AllPub,2.496911
_none_,2.531822,1Fam,2.564039,TA,2.540871,Gd,3.295455,ALQ,2.231818,⋯,CompShg,2.483601,Gable,2.418054,Normal,2.451586,Pave,2.499311,AllPub,2.496911
_none_,2.531822,1Fam,2.564039,TA,2.540871,Mn,2.675439,GLQ,3.262019,⋯,CompShg,2.483601,Gable,2.418054,Normal,2.451586,Pave,2.499311,AllPub,2.496911
_none_,2.531822,1Fam,2.564039,Gd,3.046154,No,2.320042,ALQ,2.231818,⋯,CompShg,2.483601,Gable,2.418054,Abnorml,1.920792,Pave,2.499311,AllPub,2.496911
_none_,2.531822,1Fam,2.564039,TA,2.540871,Av,2.900452,GLQ,3.262019,⋯,CompShg,2.483601,Gable,2.418054,Normal,2.451586,Pave,2.499311,AllPub,2.496911
_none_,2.531822,1Fam,2.564039,TA,2.540871,No,2.320042,GLQ,3.262019,⋯,CompShg,2.483601,Gable,2.418054,Normal,2.451586,Pave,2.499311,AllPub,2.496911


In [40]:
df <- combined_dataset_step3A %>% select(-Id, -price_log, -dataSource)

M <- abs(cor(df))
diag(M) <- 0
M <- as_data_frame(M)

cols <- purrr::map_lgl(M, function(col) { any(col > 0.8) })
names(which(cols))

character(0)

In [43]:
training_dataset <- 
    combined_dataset_step3A %>% 
    filter(dataSource == 'train') %>% 
    select(-dataSource)

report <- house_prices$model_selection$lm$greedy_r2_gain_adj(
    data = training_dataset, 
    target_var = price_log, 
    r2_gain_discard_level = 0.prcomp(USArrests, scale. = TRUE),
    r2_discard_level = 0.02, 
    a_max = 0, a_avg = 0
)

report %>% group_by(step) %>% filter(row_number() == 1)

predictor,formula,r2,step,base_r2,r2_gain,cor_abs_max,cor_abs_avg,r2_gain_adj
OverallQual,price_log ~ OverallQual,0.6747053,1,NA,NA,NA,NA,NA
GrLivArea,price_log ~ OverallQual + GrLivArea,0.7641906,2,0.6747053,13.2628602,0.6088750,0.6088750,13.2628602
Neighborhood,price_log ~ OverallQual + GrLivArea + Neighborhood,0.8132790,3,0.7641906,6.4235833,0.6790092,0.5732441,6.4235833
BsmtFinSF1,price_log ~ OverallQual + GrLivArea + Neighborhood + BsmtFinSF1,0.8493875,4,0.8132790,4.4398744,0.2426179,0.1983661,4.4398744
GarageArea,price_log ~ OverallQual + GrLivArea + Neighborhood + BsmtFinSF1 + GarageArea,0.8611900,5,0.8493875,1.3895276,0.5572300,0.4547195,1.3895276
CentralAir,price_log ~ OverallQual + GrLivArea + Neighborhood + BsmtFinSF1 + GarageArea + CentralAir,0.8702618,6,0.8611900,1.0533979,0.2725223,0.2138656,1.0533979
LotArea,price_log ~ OverallQual + GrLivArea + Neighborhood + BsmtFinSF1 + GarageArea + CentralAir + LotArea,0.8776064,7,0.8702618,0.8439511,0.3697551,0.2320913,0.8439511
YearRemodAdd,price_log ~ OverallQual + GrLivArea + Neighborhood + BsmtFinSF1 + GarageArea + CentralAir + LotArea + YearRemodAdd,0.8859860,8,0.8776064,0.9548328,0.5498273,0.3163918,0.9548328
TotalBsmtSF,price_log ~ OverallQual + GrLivArea + Neighborhood + BsmtFinSF1 + GarageArea + CentralAir + LotArea + YearRemodAdd + TotalBsmtSF,0.8914477,9,0.8859860,0.6164546,0.5385945,0.4008991,0.6164546
SaleCondition,price_log ~ OverallQual + GrLivArea + Neighborhood + BsmtFinSF1 + GarageArea + CentralAir + LotArea + YearRemodAdd + TotalBsmtSF + SaleCondition,0.8955288,10,0.8914477,0.4577993,0.3408061,0.2069990,0.4577993


In [57]:
x <- prcomp(combined_dataset_step3 %>% select(-Id, -price_log, -dataSource), scale. = TRUE)

y <- x$rotation %>% as_data_frame %>% select(PC1, PC2)

In [62]:
y

PC1,PC2
-0.064796343,0.087987427
-0.026637114,-0.322550563
-0.063089303,0.147918646
-0.089482801,0.134756628
-0.110648133,0.168303973
-0.098012588,0.197758083
0.005124755,0.084264287
-0.148968740,0.089989300
-0.067141168,0.083471838
-0.065251558,0.192025042
